# PURPOSE:

### Obtain and then prepare CLA claims with or without TPLs so that ZF transmission vouchers can be generated

Created by Daniel J. Kim - HAM Warranty Cost Analyst

## Some data quality/anomaly issues to be aware of:

- TRANS_SERIAL_NO in the WRSAL1 and WRTRN1 table has trailing white spaces.
- Missing TRANS_SERIAL_NO will cause the creation of the E-Quote part # to be invalid

## Input:

- fixed model years

## Output:

- Excel file containing initial data for non-fixed claims (CLA claims with closed TPLs)
- Excel file containing initial data for fixed claims (CLA claims without closed TPLs)

In [1]:
from datetime import datetime
from pathlib import Path
from win10toast import ToastNotifier
import numpy as np
import pandas as pd
import os
import os.path as op
import pyodbc
import scrapbook as sb
pd.options.display.max_columns=100
pd.options.display.max_rows=1000
pd.options.display.float_format = '{:20,.2f}'.format

In [2]:
username = os.environ['windowsuser']
password = os.environ['windowspwd']

### Request input parameters from user:

In [3]:
fixed_model_years = ['2015','2016']

### Obtain CLA "approved" claim records with or without TPLs:

In [4]:
%%time
cnxn_string = 'DSN=MMP-SQLP-CQP;UID=' + username + ';PWD=' + password

cnxn = pyodbc.connect(cnxn_string)
cursor = cnxn.cursor()

sql = open(r'\\mmpapp02\mq_db\wrp\ZF_ATM_WRP_Project\version4\sql\Main_Query_All_TPLs.txt').read()

try:
    print("Obtaining CLA claim records with and without TPLs.  Will take more than 5 minutes...")
    claims = pd.read_sql(sql, cnxn, parse_dates=['SUBMITTED_DATE', 'REOPENED_SUBMITTED_DATE', 'REPAIR_ORDER_DATE',
                                              'AF_OFF_DATE', 'ENGINE_BUILD_DATE'], index_col=None)

    # Close connections
    cursor.close()
    cnxn.close()
except:
    print("Error connecting to CQ server")
    cursor.close()
    cnxn.close()
print("Finished obtaining CLA claim records with and without TPLs")

Obtaining CLA claim records with and without TPLs.  May take a few minutes...
Finished obtaining CLA claim records with and without TPLs
Wall time: 7min 54s


In [5]:
claims.shape

(33931, 71)

In [6]:
claims.MODEL_YEAR.value_counts()

2016    24408
2015     7273
2017     1224
2018      816
2019      210
Name: MODEL_YEAR, dtype: int64

In [7]:
claims.head()

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM
0,5FNYF6H91GB503328001113688051,201607,5FNYF6H91GB503328,5FNYF6H91GB503328,2016-08-02,2016-08-31,NaT,NaT,2016-08-31,CLOSED,CLOSED,06200,TRANSMISSION,29.00,29.00,062015EZC05,ENGINE TEMPERATURE WENT HIGH WARNING LIGHT CAM...,Transmission was received with all bolts loose...,Removed oil cooler to perform pressure test on...,Oil Cooler - Diaphragm Damaged,ENGINE TEMPERATURE WENT HIGH WARNING LIGHT CAM...,001113,688051,ENGINE TEMPERATURE WENT HIGH WARNING LIGHT CAM...,2016-03-07,2016-07-14,2015-08-10,2015-08-07,218102,130,15A,KC,6.80,152.36,4.00,2016-07-29 14:55:23.433,100,Supplier Mfg. Defect,"9,255.00",C,"3,143.99","1,131.20",390.66,"4,665.85",N,09999,Normal,1,06200,062005EZC09RM,WARRANTY A/T KIT,1.00,GTG7,CK8,,Q5NC0072415,HONDA,2016,HMA,PILOT,9,200215NC,10.80,0.78,"2,499.65","2,049.71","2,249.69","4,099.42","2,249.69","699,502.00",1
1,5FNYF6H99GB505246001119781711,201602,5FNYF6H99GB505246,5FNYF6H99GB505246,2016-03-16,2016-07-25,2016-08-12,2016-09-08,2016-09-08,CLOSED,CLOSED,06200,TRANSMISSION,176.00,176.00,062005EZC07,CHECK FOR OIL LEAKING,Trans passed Dunk Test Cross over tube ok Out...,Functional Test ok ~ Functional Test ok,O-ring - Cut,CHECK FOR OIL LEAKING \r\nUpon further investi...,001119,781711,CHECK FOR OIL LEAKING,2015-11-28,2016-02-12,2015-10-26,2015-10-23,218102,130,15A,KC,6.80,148.92,2.60,2016-03-14 16:11:15.180,100,Supplier Mfg. Defect,40.00,C,"3,687.70",960.12,381.85,"5,029.67",N,07408,Normal,1,06200,062005EZC07RM,WARRANTY A/T KIT,1.00,GTG7,CK8,,Q5NC0093970,HONDA,2016,HMA,PILOT,9,200215NC,9.40,0.76,"2,499.65","2,598.35","2,249.69","5,196.70","2,249.69","690,344.00",1
2,5FNYF6H94GB501380001129482331,201610,5FNYF6H94GB501380,5FNYF6H94GB501380,2016-10-17,2016-12-26,2019-04-03,NaT,NaT,REOPENED_FOR_ANALYSIS,OPEN,06280,KIT,nan,918.00,062805EZC05,"DIAGNOSE DOORS NOT LOCKING, TAILGATE CLOSER NO...",~,~,None,None,001129,482331,"DIAGNOSE DOORS NOT LOCKING, TAILGATE CLOSER NO...",2016-09-09,2016-09-29,2015-06-24,2015-06-22,2231A0,130,15A,KC,0.60,0.00,0.20,2016-10-13 13:22:22.387,None,None,"16,426.00",C,"1,004.32",56.32,468.69,"1,529.33",N,03214,Normal,1,06280,062805EZC08,"CONTROL UNIT, TRANS",1.00,GTG7,CK8,,Q5NC0058838,HONDA,2016,HMA,PILOT,9,200215NC,0.80,0.75,492.14,110.76,442.93,221.52,221.52,"705,850.00",1
3,5FNYF6H90GB502851001136575701,201609,5FNYF6H90GB502851,5FNYF6H90GB502851,2016-10-12,2016-12-26,2019-04-03,NaT,NaT,REOPENED_FOR_ANALYSIS,OPEN,06200,TRANSMISSION,nan,923.00,062015EZC05,CHECK GRINDING NOIES 80-85 AND RPMS FLUCTUATIN...,~,~,None,None,001136,575701,CHECK GRINDING NOIES 80-85 AND RPMS FLUCTUATIN...,2016-07-08,2016-09-09,2015-07-23,2015-07-22,218102,130,15A,KC,6.80,704.14,8.70,2016-10-10 13:51:01.313,None,None,"13,932.00",C,"3,414.37","2,033.91",470.80,"5,919.08",N,03214,Normal,1,06200,062005EZC09RM,WARRANTY A

#### Discovered we can have VINs with missing transmission serial number (plain white space) which messes up the replaced part # formatted for equote: '20021' + SUBSTRING(WRSAL1.TRANS_SERIAL_NO,2,3) AS REPLACED_PART_EQUOTE_FMT

In [8]:
claims[claims['TRANS_SERIAL_NO'].str.startswith(' ')]

,VIN_DLR_CLM_SK,WAR_CLAIM_DATE,VIN,TPL_VIN,PICKUP_DATE,SUBMITTED_DATE,REOPENED_DATE,REOPENED_SUBMITTED_DATE,FINAL_CLOSURE_DATE,STATUS_NAME,STATUS_OPEN_CLOSED,TPL_SHORT_PART_NO,PART_CATEGORY,DAYS_TO_CLOSURE,DAYS_OPEN,WARR_PART_NO,CUSTOMER_CONTENTION_TXT,VISUAL_INSPECTION_DETAIL_TXT,FUNCTIONAL_INSPECTION_DETAIL_TXT,CATEGORY_DESC_TXT,SUPPLIER_ANALYSIS_SUMMARY_TXT,DEALER_NO,WAR_CLAIM_NO,CUS_CONT_DESC_TEXT,REPAIR_ORDER_DATE,RECEIVED_DATE,AF_OFF_DATE,ENGINE_BUILD_DATE,PRI_LABOR_OP_CODE,WAR_ORIG_DISTR_CDE,WAR_RESP_DISTR_CDE,VEHICLE_DESTN_CODE,FLAT_RATE_HRS_QTY,FRGT_SUBL_TAX_AMT_USD,DIAGNOSTIC_LABOR_HRS_QTY,TPL_CREATION_DATE,RESPONSIBILITY_PCT,CONCLUSION_DESC_TXT,VEH_MILEAGE,CURRENCY_TYPE_CODE,ACTUAL_PARTS_CHG_AMT_USD,ACTUAL_LABOR_CHG_AMT_USD,ACTUAL_HDLG_CHG_AMT_USD,ACTUAL_TOTAL_CHG_AMT_USD,CAMPAIGN_CODE,WAR_DEFECT_CODE,RECORD_DESC,RECORD_COUNT,REPLACED_PART5,REPLACED_PART_NO,PART_DESC,ACT_PARTS_QTY_NO,MTC_MODEL,MTC_TYPE,MTC_OPTION,TRANS_SERIAL_NO,NAMEPLATE,MODEL_YEAR,FACTORY,MOD_NAME,TRANSMISSION_GEARS,REPLACED_PART_EQUOTE_FMT,CLAIM_LABOR_HRS_QTY,WAR_EXCH_RATE_AMT,DLR_NET_PRCE_AMT_USD,PART_FOB_COST_USD,DLR_NET_PRCE_AMTx90_USD,PART_FOB_COSTx2_USD,MIN_PART_COST_USD,TRANSFER_PART_LIST_ITEM_DETAIL_SK,REPL_ROW_NUM
21200,5FRYD4H92GB022416251336392305,201801,5FRYD4H92GB022416,None,NaT,NaT,NaT,NaT,NaT,None,None,None,None,nan,nan,062005NBA58,None,~,~,None,None,251336,392305,Zf 9 a/t trans warmer pud,2017-12-20,2018-01-05,2015-05-06,2015-05-05,2180A7,101,15A,KA,2.10,0.00,0.00,NaT,None,None,"29,627.00",$,30.56,230.75,27.08,288.39,C,6ZF00,Normal,1,06224,062245J4305 ...,"KIT, OIL COOLER",1.00,GTZ6,AE5,,,ACURA,2016,HMA,MDX,9,20021,2.10,1.00,39.78,15.91,35.80,31.82,31.82,nan,1


In [9]:
claims[claims['TRANS_SERIAL_NO'].str.startswith(' ')]['TRANS_SERIAL_NO'].values

array(['                    '], dtype=object)

In [10]:
# This reads as: give me rows where replaced part equote = '20021   '' and bring back just the replaced_part_equote_fmt column:
claims.loc[claims['REPLACED_PART_EQUOTE_FMT']=='20021   ', 'REPLACED_PART_EQUOTE_FMT']

21200    20021   
Name: REPLACED_PART_EQUOTE_FMT, dtype: object

In [11]:
claims.loc[claims['REPLACED_PART_EQUOTE_FMT']=='20021   ', 'REPLACED_PART_EQUOTE_FMT'].values

array(['20021   '], dtype=object)

### Convert claims with bad E-Quote part # format as NaN and then forward fill missing E-Quote replaced part #

In [12]:
claims.loc[claims['REPLACED_PART_EQUOTE_FMT'] == '20021   ', 'REPLACED_PART_EQUOTE_FMT'] = np.nan
claims.sort_values(by=['MOD_NAME', 'MODEL_YEAR', 'AF_OFF_DATE'], inplace=True)
claims['REPLACED_PART_EQUOTE_FMT'].fillna(method='ffill', inplace=True)

In [13]:
assert len(claims.loc[claims['REPLACED_PART_EQUOTE_FMT']=='20021   ', 'REPLACED_PART_EQUOTE_FMT']) == 0

### TRANS_SERIAL_NO has trailing white spaces, so need to remove them so that we can merge with ASN data

In [14]:
claims['TRANS_SERIAL_NO'].values

array(['Q5NC0011360         ', 'Q5NC0011388         ',
       'Q5NC0011324         ', ..., 'Q5J40456287         ',
       'Q5L90575501         ', 'Q5J40435973         '], dtype=object)

In [15]:
claims['TRANS_SERIAL_NO'] = claims['TRANS_SERIAL_NO'].str.strip()

# Now limit data set to just claims with closed TPLs and save as Excel file:

In [16]:
criteria1 = claims['TRANSFER_PART_LIST_ITEM_DETAIL_SK'].isna()
criteria2 = claims['MODEL_YEAR'].isin(fixed_model_years)

In [17]:
claims_with_tpl = claims[~criteria1 & ~criteria2]

In [18]:
claims_with_tpl.shape

(1479, 71)

In [19]:
today = datetime.today()

In [20]:
base_dir = "//mmpapp02/mq_db/wrp/ZF_ATM_WRP_Project/automated_vouchers/outputs/excel/NorthAmericanPlants/NonFixed"
p = Path(base_dir)
save_dir = p / datetime.strftime(today, "%Y-%m-%d")
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [21]:
file_name_nonfixed = 'CLA_Claims_With_TPLs.xlsx'

In [22]:
claims_with_tpl.to_excel(save_dir / file_name_nonfixed, index=False)

In [23]:
str(save_dir / file_name_nonfixed)

'\\\\mmpapp02\\mq_db\\wrp\\ZF_ATM_WRP_Project\\automated_vouchers\\outputs\\excel\\NorthAmericanPlants\\NonFixed\\2019-04-23\\CLA_Claims_With_TPLs.xlsx'

### Now save/"glue" the save location or path to the Excel file containing CLA claims with closed TPL:

In [24]:
sb.glue("path_to_claims_with_tpl", str(save_dir / file_name_nonfixed))

# Prepare data for fixed voucher (CLA claims without closed TPL):

In [25]:
base_dir = "//mmpapp02/mq_db/wrp/ZF_ATM_WRP_Project/automated_vouchers/outputs/excel/NorthAmericanPlants/Fixed"
p = Path(base_dir)
save_dir = p / datetime.strftime(today, "%Y-%m-%d")
if not os.path.exists(save_dir):
    os.makedirs(save_dir)

In [26]:
# Claims without TPLs
criteria1 = (claims['TRANSFER_PART_LIST_ITEM_DETAIL_SK'].isna())

# Claims with TPLs, but not closed (in "limbo")
criteria2 = (claims['CONCLUSION_DESC_TXT'].isna()) & claims['TRANSFER_PART_LIST_ITEM_DETAIL_SK'] > 0

# Claims from fixed model years
criteria3 = (claims['MODEL_YEAR'].isin(fixed_model_years))

# "Normal" claims
criteria4 = (claims['CAMPAIGN_CODE'] == 'N')

In [27]:
fixed_claims = claims[(criteria1 | criteria2) & criteria3 & criteria4]

In [28]:
fixed_claims.shape

(4399, 71)

In [29]:
file_name_fixed = 'CLA_Claims_Fixed.xlsx'

In [30]:
fixed_claims.to_excel(save_dir / file_name_fixed, index=False)

In [31]:
str(save_dir / file_name_fixed)

'\\\\mmpapp02\\mq_db\\wrp\\ZF_ATM_WRP_Project\\automated_vouchers\\outputs\\excel\\NorthAmericanPlants\\Fixed\\2019-04-23\\CLA_Claims_Fixed.xlsx'

### Now save/"glue" the save location or path to the Excel file containing fixed CLA claims:

In [32]:
sb.glue("path_to_fixed_claims", str(save_dir / file_name_fixed))

# Send Windows notification when complete

In [33]:
toaster = ToastNotifier()
toaster.show_toast("Papermill Orchestration Status",
                   "Successfully provided data sets for voucher creation",
                   icon_path="images/honda_logo.ico",
                   duration=5)

True